In [ ]:

def get_cmc_session():
    s = requests.Session()
    s.headers.update({
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/json",
        "Origin": "https://coinmarketcap.com",
        "Referer": "https://coinmarketcap.com/events/",
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/139.0.0.0 Safari/537.36"),
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Platform": "web",
    })

    # 이벤트 페이지 방문해서 토큰 확보
    r = s.get("https://coinmarketcap.com/events/", timeout=5)

    csrf = s.cookies.get("x-csrf-token")
    if not csrf:
        raise RuntimeError("x-csrf-token을 못 가져왔습니다. 응답 확인 필요")

    # 헤더와 쿠키 둘 다 반영
    s.headers["x-csrf-token"] = csrf
    s.cookies.set("x-csrf-token", csrf, domain=".coinmarketcap.com")

    return s

# 사용 예시
if __name__ == "__main__":
    session = get_cmc_session()
    print("CSRF token:", session.headers["x-csrf-token"])


In [1]:
import requests

In [3]:
s = requests.Session()
s.headers.update({
    "Accept": "application/json, text/plain, */*",
    "Content-Type": "application/json",
    "Origin": "https://coinmarketcap.com",
    "Referer": "https://coinmarketcap.com/events/",
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                    "AppleWebKit/537.36 (KHTML, like Gecko) "
                    "Chrome/139.0.0.0 Safari/537.36"),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Platform": "web",
})

In [4]:
# 이벤트 페이지 방문해서 토큰 확보
r = s.get("https://coinmarketcap.com/events/", timeout=5)

csrf = s.cookies.get("x-csrf-token")

In [7]:
s.cookies

<RequestsCookieJar[]>

In [8]:
import requests

s = requests.Session()
s.headers.update({
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cache-Control": "no-cache",
    "Pragma": "no-cache",
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/139.0.0.0 Safari/537.36"),
    "Upgrade-Insecure-Requests": "1",
    # client hints & fetch metadata
    "sec-ch-ua": '"Not;A=Brand";v="99", "Google Chrome";v="139", "Chromium";v="139"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
})

def show(resp, tag):
    print(f"[{tag}] status={resp.status_code}")
    print("Set-Cookie:", resp.headers.get("Set-Cookie"))
    print("Cookies:", s.cookies.get_dict())

r1 = s.get("https://coinmarketcap.com/", timeout=10, allow_redirects=True)
show(r1, "home")

# 새 페이지에서 fetch 메타데이터를 cross-site 로 맞춤
s.headers.update({
    "Referer": "https://coinmarketcap.com/",
    "Origin": "https://coinmarketcap.com",
    "sec-fetch-site": "same-origin",
    "sec-fetch-mode": "navigate",
    "sec-fetch-dest": "document",
})

r2 = s.get("https://coinmarketcap.com/events/", timeout=10, allow_redirects=True)
show(r2, "events")

csrf = s.cookies.get("x-csrf-token")
print("x-csrf-token =", csrf)


[home] status=200
Set-Cookie: None
Cookies: {}
[events] status=200
Set-Cookie: None
Cookies: {}
x-csrf-token = None


In [2]:
# pip install playwright requests
# python -m playwright install chromium
import asyncio, requests, json
from playwright.async_api import async_playwright

STORAGE="cmc_storage.json"

async def harvest():
    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=True)
        ctx = await browser.new_context(storage_state=STORAGE if False else None,
                                        user_agent=("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                                    "AppleWebKit/537.36 (KHTML, like Gecko) "
                                                    "Chrome/139.0.0.0 Safari/537.36"))
        page = await ctx.new_page()
        await page.goto("https://coinmarketcap.com/events/", wait_until="domcontentloaded")
        # JS로 쿠키 세팅될 시간을 약간 줌
        await page.wait_for_timeout(2000)
        state = await ctx.storage_state(path=STORAGE)  # 상태 저장
        await browser.close()
        return json.loads(state)["cookies"]

def requests_session_from_cookies(cookies):
    s = requests.Session()
    for c in cookies:
        s.cookies.set(c["name"], c["value"], domain=c.get("domain") or ".coinmarketcap.com")
    return s

async def main():
    cookies = await harvest()
    s = requests_session_from_cookies(cookies)
    csrf = s.cookies.get("x-csrf-token")
    if not csrf:
        raise RuntimeError("x-csrf-token이 여전히 없습니다. (안티봇/캡차 가능성)")

    s.headers.update({
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/json",
        "Origin": "https://coinmarketcap.com",
        "Referer": "https://coinmarketcap.com/events/",
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/139.0.0.0 Safari/537.36"),
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Platform": "web",
        "x-csrf-token": csrf,
    })

    payload = {"startDate":"2025-09-10","endDate":"2025-09-24","page":1,"size":20}
    r = s.post("https://api.coinmarketcap.com/data-api/v3/calendar/query", json=payload, timeout=20)
    print(r.status_code, r.text[:500])

asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop